In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

import pandas as pd
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None
import numpy as np
import random

import matplotlib.pyplot as plt
import seaborn as sns
sns.set('talk')
sns.set_style('white')

import imgaug as ia
from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications.efficientnet import EfficientNetB4, preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
randomSeed = 42

ia.seed(randomSeed)

seq = iaa.Sequential(
    [
        iaa.Affine(rotate=(-15, 15)),
        iaa.Fliplr(0.5),
        iaa.GaussianBlur((0, 2.0)),
        iaa.ElasticTransformation(alpha=(0, 70), sigma=9),
        iaa.AdditiveGaussianNoise(scale=(0, 0.05), per_channel=True),
        iaa.ChannelShuffle(p=0.5),
    ],
    random_order=False,
)

class CutMixImageDataGenerator():
    def __init__(self, generator1, generator2, img_size, batch_size):
        self.batch_index = 0
        self.samples = generator1.samples
        self.class_indices = generator1.class_indices
        self.generator1 = generator1
        self.generator2 = generator2
        self.img_size = img_size
        self.batch_size = batch_size

    def reset_index(self):  # Ordering Reset (If Shuffle is True, Shuffle Again)
        self.generator1._set_index_array()
        self.generator2._set_index_array()

    def reset(self):
        self.batch_index = 0
        self.generator1.reset()
        self.generator2.reset()
        self.reset_index()

    def get_steps_per_epoch(self):
        quotient, remainder = divmod(self.samples, self.batch_size)
        return (quotient + 1) if remainder else quotient
    
    def __len__(self):
        self.get_steps_per_epoch()

    def __next__(self):
        if self.batch_index == 0: self.reset()

        crt_idx = self.batch_index * self.batch_size
        if self.samples > crt_idx + self.batch_size:
            self.batch_index += 1
        else:  # If current index over number of samples
            self.batch_index = 0

        reshape_size = self.batch_size
        last_step_start_idx = (self.get_steps_per_epoch()-1) * self.batch_size
        if crt_idx == last_step_start_idx:
            reshape_size = self.samples - last_step_start_idx
            
        X_1, y_1 = self.generator1.next()
        X_2, y_2 = self.generator2.next()
        
        cut_ratio = np.random.beta(a=1, b=1, size=reshape_size)
        cut_ratio = np.clip(cut_ratio, 0.2, 0.8)
        label_ratio = cut_ratio.reshape(reshape_size, 1)
        cut_img = X_2

        X = X_1
        for i in range(reshape_size):
            cut_size = int((self.img_size-1) * cut_ratio[i])
            y1 = random.randint(0, (self.img_size-1) - cut_size)
            x1 = random.randint(0, (self.img_size-1) - cut_size)
            y2 = y1 + cut_size
            x2 = x1 + cut_size
            cut_arr = cut_img[i][y1:y2, x1:x2]
            cutmix_img = X_1[i]
            cutmix_img[y1:y2, x1:x2] = cut_arr
            X[i] = cutmix_img
            
        X = seq.augment_images(X)  # Sequential of imgaug
        y = y_1 * (1 - (label_ratio ** 2)) + y_2 * (label_ratio ** 2)
        return X, y

    def __iter__(self):
        while True:
            yield next(self)

In [ ]:
trainDF = pd.read_csv('../train.csv')
trainingDirectory = '../trainAllResized'
target = trainDF['category'].apply(lambda c: str(c).zfill(2))
noClasses = len(np.unique(target))
imageWidth = 229
imageHeight = 229
imageDimension = (imageWidth, imageHeight)
batchSize = 16
validationRatio = 0.2
preprocess = preprocess_input
    
X_train, X_val, y_train, y_val = train_test_split(trainDF[['filename']], target, 
                                                  test_size=validationRatio, random_state=randomSeed,
                                                  stratify=target)
train = pd.concat([X_train, y_train], axis=1)
val = pd.concat([X_val, y_val], axis=1)

trainDataGenerator = ImageDataGenerator(horizontal_flip=True,
                                        brightness_range=[0.6, 1.4],
                                        preprocessing_function=preprocess)

trainGenerator1 = trainDataGenerator.flow_from_dataframe(
                dataframe=train,
                directory=trainingDirectory,
                x_col='filename',
                y_col='category',
                target_size=imageDimension,
                batch_size=batchSize,
                class_mode='categorical',
                shuffle=True)

trainGenerator2 = trainDataGenerator.flow_from_dataframe(
                dataframe=train,
                directory=trainingDirectory,
                x_col='filename',
                y_col='category',
                target_size=imageDimension,
                batch_size=batchSize,
                class_mode='categorical',
                shuffle=True)

# CutMixImageDataGenerator
trainGenerator = CutMixImageDataGenerator(
    generator1=trainGenerator1,
    generator2=trainGenerator2,
    img_size=imageDimension[0],
    batch_size=batchSize)


valDataGenerator = ImageDataGenerator(preprocessing_function=preprocess)

valGenerator = valDataGenerator.flow_from_dataframe(
            dataframe=val,
            directory=trainingDirectory,
            x_col='filename',
            y_col='category',
            target_size=imageDimension,
            batch_size=batchSize,
            class_mode='categorical',
            shuffle=False)

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
               'balanced',
                classes=np.unique(trainGenerator1.classes), 
                y=trainGenerator1.classes)

class_weight = dict(enumerate(class_weights))

In [ ]:
baseModel = EfficientNetB4(include_top=False, weights='imagenet', input_shape=(imageWidth, imageHeight, 3))
model = Sequential()
model.add(baseModel)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(1e-3)))
model.add(Dropout(0.5))
model.add(Dense(noClasses, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=5e-05),
              metrics=['acc'])

model.load_weights('checkpoint/model.18-0.8034-0.8112-0.7413.h5')

model.summary()

In [ ]:
checkpoint_filepath = 'checkpoint/model.{epoch:02d}-{val_loss:.4f}-{val_acc:.4f}-{acc:.4f}.h5'
checkpointCallback = ModelCheckpoint(filepath=checkpoint_filepath, 
                                            save_weights_only=True, 
                                            monitor='val_acc',
                                            mode='max',
                                            save_best_only=True)

earlyStop = EarlyStopping(monitor='val_acc', mode='max', patience=16, verbose=1)

reduceLR = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=4, min_lr=1e-6, verbose=1)

history = model.fit(trainGenerator, 
                  steps_per_epoch=trainGenerator1.n//trainGenerator1.batch_size, 
                  epochs=100,
                  validation_data=valGenerator, 
                  validation_steps=valGenerator.n//valGenerator.batch_size,
                  verbose=1,
                  class_weight=class_weight,
                  initial_epoch=18,
                  callbacks=[checkpointCallback, earlyStop, reduceLR])